In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from einops import parse_shape

sequence_length = 100 # L
batch_size = 10 # B
input_size = 15 # N
hidden_size = 20 # H == output_size

rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=1)

input = torch.randn(sequence_length, batch_size, input_size)
h0 = torch.randn(1, batch_size, hidden_size)
output, hn = rnn(input, h0)

out_shape = parse_shape(output, 'L B H')
hn_shape = parse_shape(hn, '_ B H')

print(out_shape, hn_shape)

# Good thing is that we can provide inputs of different lengths.
# Bad thing is that the entire provided sequence is computed sequentially. 

NameError: name 'nn' is not defined

In [ ]:
k = 1024 # vocab size = number of symbols = num_embeddings
d = 64 # embedding_dim 

key = nn.Linear(in_features = d, out_features = d)

emb = nn.Embedding(k, d)
x = emb(torch.tensor([1, 100, 999, 55]))
print(x.shape) # the length of the input is L=4 <=! num_blocks = block_size

x = rearrange(x, 'L D -> 1 L D')

print(x.shape)

k = key(x)

print(k.shape)

num_heads = 2
k = rearrange(k, 'b s (num_heads heads_dim) -> b num_heads s heads_dim', num_heads = num_heads)
print(k.shape)



In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from brainle.models.architectures.attention import CausalSelfAttention

net = CausalSelfAttention(
    embedding_dim = 64,
    num_heads = 4, 
    block_size = 100,
    dropout_attention = 0.5,
    dropout_residual = 0.5
)

net(torch.rand(1, 100, 64)).shape

torch.Size([1, 100, 64])

In [99]:
torch.sqrt(torch.tensor(5))

tensor(2.2361)

In [145]:
torch.einsum('ik, k -> i', torch.tensor([[1,2],[3,4]]), torch.tensor([2,2]))

tensor([ 6, 14])